In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from itertools import chain
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import numpy as np
import re
from numpy import random

In [52]:
#all_countries=pd.read_csv("C:/Users/homa/Python-Notbook/TradeMap/all_countries.csv")
Product_Code=pd.read_csv("C:/Users/homa/Python-Notbook/TradeMap/Product_Codem.csv")
#all_countries.head(2)
Product_Code.drop_duplicates(subset=['ProductCode'], inplace=True)

In [39]:
driver= webdriver.Chrome()
driver.get("https://www.trademap.org/Index.aspx")
#driver.maximize_window()
waits= WebDriverWait(driver, 30)

Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


In [40]:
def login():
    try:
        
        logintext=driver.find_element(By.CSS_SELECTOR ,("#ctl00_MenuControl_Label_Login"))
        if logintext.text=='Login': 
                
            login=waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_MenuControl_Label_Login')))
            login.click()
            url= driver.current_url
            if url!='https://www.trademap.org/Index.aspx': 
                getemail= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#Username')))
                getemail.send_keys('homa.abedinzadeh@gmail.com')
                #time.sleep(random.randint(5,20))
                getpassword= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#Password')))
                getpassword.send_keys('Aa@97019701')
                #time.sleep(random.randint(5,20))
                waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.container > div > div:nth-child(2) > div > div > div > form > fieldset > div:nth-child(4) > div > button'))).click()
            
    
    except Exception as e:
                print(f"Error selecting option : {e}")


In [41]:
def delete():
    waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Image_deleteProduct'))).click()
    waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Image_DeleteCountry'))).click()  

In [56]:
df = pd.DataFrame()
temp = pd.DataFrame()
for i in range(len(Product_Code)):
    
    login()
    time.sleep(random.randint(10,15))
    delete()
    time.sleep(random.randint(4,7))
    inputproduct = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_Input')))
    inputproduct.send_keys(Product_Code['ProductCode'].iloc[i])
    time.sleep(random.randint(5,8))
    proclick = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_c0'))).click()
    time.sleep(random.randint(6,9))
    btn = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Button_TradeIndicators'))).click()
    logintext=driver.find_element(By.CSS_SELECTOR ,("#ctl00_MenuControl_Label_Login"))
    if logintext.text=='Login':  
        login()


    dropdown_element = waits.until(EC.visibility_of_element_located((By.ID, 'ctl00_NavigationControl_DropDownList_Country')))  # Replace with actual ID     
    select = Select(dropdown_element)
    options = select.options
    j=0
    while True:  
        print(j)
        try:
            
            dropdown_element = waits.until(EC.visibility_of_element_located((By.ID, 'ctl00_NavigationControl_DropDownList_Country')))
            select = Select(dropdown_element)
            options = select.options
            option_text = options[j].text

            if option_text != 'All' or option_text.find('elsewhere ')!=-1  :
                select.select_by_visible_text(option_text)
                time.sleep(random.randint(2,3))
                # تغییر تعداد رکوردها به 300
                incshow = waits.until(EC.element_to_be_clickable((By.XPATH, "//select[@id='ctl00_PageContent_GridViewPanelControl_DropDownList_PageSize']")))
                
                drop = Select(incshow)
                
                time.sleep(random.randint(2,3))
                drop.select_by_value('300')





            # استخراج جدول
                all_tables = pd.read_html(driver.page_source, attrs={'id': 'ctl00_PageContent_MyGridView1'})
                temp = all_tables[0]

                if temp.shape[0] != 1:
                    temp = temp.iloc[1:, 1:].iloc[:, :-4]
                    new_header = temp.iloc[0]
                    temp = temp[1:]
                    temp.columns = new_header

                    # افزودن اطلاعات کشور و کد محصول
                    temp['Importer'] = option_text
                    temp['productcode'] = re.sub(r'[^\w]', '', Product_Code['ProductCode'].iloc[i])
                    temp.drop(temp[temp['Exporters'] == 'Total'].index, inplace=True)
                    temp.drop(temp[temp['Exporters'] == 'World'].index, inplace=True)

                    # تغییر نام ستون‌ها
                    average = temp.columns[15]
                    sharein = temp.columns[3]
                    temp.rename(columns={average: 'Average tariff%', sharein: 'ShareinCounryImports'}, inplace=True)

                    # افزودن به DataFrame اصلی
                    df = pd.concat([df, temp], axis=0)
                    temp = pd.DataFrame()

                    print(f"Selected option: {option_text}, {j}")
                time.sleep(random.randint(1,3))
            if (j+1)%10 ==0:
                    #driver.quit()
                    #driver= webdriver.Chrome()
                    waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_MenuControl_Label_Menu_Home'))).click()
                    time.sleep(random.randint(5,8))
                    login()
                    delete()
                    time.sleep(random.randint(4,7))
                    inputproduct = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_Input')))
                    inputproduct.send_keys(Product_Code['ProductCode'].iloc[i])
                    time.sleep(random.randint(5,8))
                    proclick = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_c0'))).click()
                    time.sleep(random.randint(6,9))
                    btn = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Button_TradeIndicators'))).click()
                     
        
        except Exception as e:
            print(f"Error selecting option {j}: {e}")
            site = driver.current_url
            if site!= 'https://www.trademap.org/Index.aspx':
                driver.get('https://www.trademap.org/Index.aspx')
            login()
            delete()
            time.sleep(random.randint(4,7))
            inputproduct = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_Input')))
            inputproduct.send_keys(Product_Code['ProductCode'].iloc[i])
            time.sleep(random.randint(5,8))
            proclick = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_c0'))).click()
            time.sleep(random.randint(6,9))
            btn = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Button_TradeIndicators'))).click()
            if len (df[(df.Importer==option_text) & (df.productcode==re.sub(r'[^\w]', '', Product_Code['ProductCode'].iloc[i])) ])==0:
                j-=1
            
            
        
        j+=1
        if j+1> len (options):
            print (f"Get All Country this product:{Product_Code['ProductCode'].iloc[i]}")
            break
    waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_MenuControl_Label_Menu_Home'))).click()
    delete()
    time.sleep(30)
    #driver.refresh()
    
    driver.get('https://www.trademap.org/Index.aspx')
    i+=1
print ('All Requests')

0
1
Selected option: Afghanistan, 1
2
3
Selected option: Albania, 3
4
Selected option: Algeria, 4
5
6
7
Selected option: Andorra, 7
8
Selected option: Angola, 8
9
Selected option: Anguilla, 9
10
Selected option: Antigua and Barbuda, 10
11
Selected option: Area Nes, 11
12
Selected option: Argentina, 12
13
Selected option: Armenia, 13
14
Selected option: Aruba, 14
15
16
Selected option: Australia, 16
17
Selected option: Austria, 17
18
Selected option: Azerbaijan, 18
19
Selected option: Bahamas, 19
20
Selected option: Bahrain, 20
21
Selected option: Bangladesh, 21
22
Selected option: Barbados, 22
23
Selected option: Belarus, 23
24
Selected option: Belgium, 24
25
Selected option: Belize, 25
26
Selected option: Benin, 26
27
Selected option: Bermuda, 27
28
Selected option: Bhutan, 28
29
Selected option: Bolivia, Plurinational State of, 29
30
Selected option: Bonaire, Sint Eustatius and Saba, 30
31
Selected option: Bosnia and Herzegovina, 31
32
Selected option: Botswana, 32
33
34
Selected opt

In [ ]:
df.to_csv('C:/Users/homa/Python-Notbook/TradeMap/df.csv')

In [51]:
x.to_csv('C:/Users/homa/Python-Notbook/TradeMap/df.csv', mode='a', index=False, header=False)